# 6_GWAS_enrichment

from Yang

shuffling files and then running enrichment in each tissue




# Step 1: sbatch to bed intersect with diffloop atac regions

```
#!/bin/bash
#
#SBATCH --job-name=diffLoops_allATAC_Loops_psych
#SBATCH --output=/oak/stanford/groups/khavari/users/yzhao8/Margaret/GWAS/logs/diffLoops_allATAC_Loops_psych.out
#SBATCH --error=/oak/stanford/groups/khavari/users/yzhao8/Margaret/GWAS/logs/diffLoops_allATAC_Loops_psych.err
#SBATCH --time=48:00:00
#SBATCH --partition=khavari,rbaltman,normal,owners
#SBATCH --ntasks=1
#SBATCH --mem=100G
#SBATCH --cpus-per-task=16
#SBATCH --mail-type=FAIL,END
#SBATCH --mail-user=mguo123@stanford.edu

cd /oak/stanford/groups/khavari/users/mguo123/psych_project/data/GWAS_enrichment
ml biology bedtools

###### STEP 1: make diff atac files from diffloop bedpe and atac bed files
# (base) [mguo123@sh02-10n18 /oak/stanford/groups/khavari/users/mguo123/psych_project/data/atac_bed_files]$ ls
# Astrocytes_merged.bed  H9_D0_merged.bed  H9_D10_merged.bed  H9_D28_merged.bed  H9_D2_merged.bed  H9_D4_merged.bed  old_files  SLC_D0_merged.bed  SLC_D2_merged.bed  SL_D0_merged.bed  SL_D2_merged.bed


# (base) [mguo123@sh02-10n18 /oak/stanford/groups/khavari/users/mguo123/psych_project/data/GWAS_enrichment/diffloop_allatac_anchors]$ ls  ../../hichip_diffloop_files/*anchor.bed
# ../../hichip_diffloop_files/Astro_diffloop_anchor.bed  ../../hichip_diffloop_files/H9_D10_diffloop_anchor.bed  ../../hichip_diffloop_files/H9_D2_diffloop_anchor.bed   ../../hichip_diffloop_files/SLC_D2_diffloop_anchor.bed  ../../hichip_diffloop_files/SL_D2_diffloop_anchor.bed
# ../../hichip_diffloop_files/H9_D0_diffloop_anchor.bed  ../../hichip_diffloop_files/H9_D28_diffloop_anchor.bed  ../../hichip_diffloop_files/SLC_D0_diffloop_anchor.bed  ../../hichip_diffloop_files/SL_D0_diffloop_anchor.bed

# tissue="Astro"
# tissue="H9_D0" 
# tissue="H9_D10" 
# tissue="H9_D28" 
# tissue="H9_D2" 
# tissue="SL_D0" 
# tissue="SL_D2" 
# tissue="SLC_D0" 
# tissue="SLC_D2" 

# tissue="SL_D2" 
# atac_tissue=${tissue}
# #except Astrocytes!!!
# cd /oak/stanford/groups/khavari/users/mguo123/psych_project/data/GWAS_enrichment/diffloop_allatac_anchors
# awk '{print "chr"$1"\t"$2"\t"$3}' ../../hichip_diffloop_files/${tissue}_diffloop_anchor.bed | sort  -k1,1 -k2,2n| uniq |  bedtools intersect -a stdin -b ../../atac_bed_files/${atac_tissue}_merged.bed -wa | uniq > ${tissue}_diffloop_atac_anchor.bed 
# cd ../diffloop_allatac_loop
# awk '{print "chr"$1"\t"$2"\t"$3"\t""chr"$4"\t"$5"\t"$6"\t"$7}' ../../hichip_diffloop_files/${tissue}_diffloop_loop.bed | bedtools intersect -a stdin -b ../diffloop_allatac_anchors/${tissue}_diffloop_atac_anchor.bed  -wa > ${tissue}_diffloop_atac_loop.bed 


# (base) [mguo123@sh02-10n18 /oak/stanford/groups/khavari/users/mguo123/psych_project/data/GWAS_enrichment]$ wc -l diffloop_allatac_anchors/*
#   13952 diffloop_allatac_anchors/Astro_diffloop_atac_anchor.bed
#   32687 diffloop_allatac_anchors/H9_D0_diffloop_atac_anchor.bed
#   19653 diffloop_allatac_anchors/H9_D10_diffloop_atac_anchor.bed
#   13694 diffloop_allatac_anchors/H9_D28_diffloop_atac_anchor.bed
#       0 diffloop_allatac_anchors/H9_D2_diffloop_atac_anchor.bed
#   40071 diffloop_allatac_anchors/SLC_D0_diffloop_atac_anchor.bed
#       0 diffloop_allatac_anchors/SLC_D2_diffloop_atac_anchor.bed
#   38638 diffloop_allatac_anchors/SL_D0_diffloop_atac_anchor.bed
#       0 diffloop_allatac_anchors/SL_D2_diffloop_atac_anchor.bed
#  158695 total

## NOTE: SKIPP INg the NGN2s

# ####### STEP 2: make blacklist exclusion files
# cd /oak/stanford/groups/khavari/users/mguo123/psych_project/data/GWAS_enrichment
# tissues=( Astro  H9_D0 H9_D10 H9_D28 H9_D2 SL_D0 SL_D2 SLC_D0 SLC_D2)
# for tissue in "${tissues[@]}";
# do
# echo $tissue
# cat ./resources/hg19.blackList.bed ./diffloop_allatac_anchors/${tissue}_diffloop_atac_anchor.bed  > ./diffloop_atac_anchors_exclude/${tissue}_excl.bed
# done


###### STEP 3: shuffling
#results dir
mkdir -p result_diffloop_atac

cd /oak/stanford/groups/khavari/users/mguo123/psych_project/data/GWAS_enrichment
tissues=( Astro  H9_D0 H9_D10 H9_D28 SL_D0  SLC_D0)
for tissue in "${tissues[@]}";
do
echo $tissue

#REAL THING
 cut -f1-3 ./diffloop_allatac_loop/${tissue}_diffloop_atac_loop.bed > ./result_diffloop_atac/${tissue}_ACTUAL_anchor1.bed
 cut -f4-6 ./diffloop_allatac_loop/${tissue}_diffloop_atac_loop.bed > ./result_diffloop_atac/${tissue}_ACTUAL_anchor2.bed
 cat ./result_diffloop_atac/${tissue}_ACTUAL_anchor1.bed ./result_diffloop_atac/${tissue}_ACTUAL_anchor2.bed > ./result_diffloop_atac/${tissue}_ACTUAL_anchors.bed
 rm ./result_diffloop_atac/${tissue}_ACTUAL_anchor1.bed
 rm ./result_diffloop_atac/${tissue}_ACTUAL_anchor2.bed
bedtools intersect -a ./resources/LinkedSNP-all-sorted.bed -b ./diffloop_allatac_anchors/${tissue}_diffloop_atac_anchor.bed -wa | sort -k1,1 -k2,2n | uniq > ./result_diffloop_atac/${tissue}_ACTUAL_overlap_all.bed
bedtools intersect -a ./resources/psych_mpra_lib.bed -b ./diffloop_allatac_anchors/${tissue}_diffloop_atac_anchor.bed -wa | sort -k1,1 -k2,2n | uniq > ./result_diffloop_atac/${tissue}_ACTUAL_overlap_psychmpra.bed
bedtools intersect -a ./resources/SNP-nonCancer-nonFilter-1.bed  -b ./diffloop_allatac_anchors/${tissue}_diffloop_atac_anchor.bed -wa | sort -k1,1 -k2,2n | uniq > ./result_diffloop_atac/${tissue}_ACTUAL_overlap_noncancer.bed
 rm ./result_diffloop_atac/${tissue}_ACTUAL_anchors.bed

#SHUFFLED THING
for m in {1..100}
do
 bedtools shuffle -i ./diffloop_allatac_loop/${tissue}_diffloop_atac_loop.bed -g ./resources/chrom_hg19.sizes -seed $m -excl ./diffloop_atac_anchors_exclude/${tissue}_excl.bed -bedpe > ./result_diffloop_atac/${tissue}_"$m".bed
 cut -f1-3 ./result_diffloop_atac/${tissue}_"$m".bed > ./result_diffloop_atac/${tissue}_"$m"_anchor1.bed
 cut -f4-6 ./result_diffloop_atac/${tissue}_"$m".bed > ./result_diffloop_atac/${tissue}_"$m"_anchor2.bed
 cat ./result_diffloop_atac/${tissue}_"$m"_anchor1.bed ./result_diffloop_atac/${tissue}_"$m"_anchor2.bed > ./result_diffloop_atac/${tissue}_"$m"_anchors.bed
 rm ./result_diffloop_atac/${tissue}_"$m"_anchor1.bed
 rm ./result_diffloop_atac/${tissue}_"$m"_anchor2.bed
 bedtools intersect -a ./resources/LinkedSNP-all-sorted.bed -b ./result_diffloop_atac/${tissue}_"$m"_anchors.bed -wa | sort -k1,1 -k2,2n | uniq > ./result_diffloop_atac/${tissue}_"$m"_overlap_all.bed
 bedtools intersect -a ./resources/psych_mpra_lib.bed -b ./result_diffloop_atac/${tissue}_"$m"_anchors.bed -wa | sort -k1,1 -k2,2n | uniq > ./result_diffloop_atac/${tissue}_"$m"_overlap_psychmpra.bed
 bedtools intersect -a ./resources/SNP-nonCancer-nonFilter-1.bed  -b ./result_diffloop_atac/${tissue}_"$m"_anchors.bed -wa | sort -k1,1 -k2,2n | uniq > ./result_diffloop_atac/${tissue}_"$m"_overlap_noncancer.bed
 rm ./result_diffloop_atac/${tissue}_"$m"_anchors.bed
#done shuffle loop
done


# done tissues loop
done

```

# Step 2: take shuffled bed files and make a single .txt file per tissue per snp bed file tested (using R on sherlock)
then scp over to `gwas_enrichment_shuffle`

`scp mguo123@login.sherlock.stanford.edu:/oak/stanford/groups/khavari/users/mguo123/psych_project/data/GWAS_enrichment/result_diffloop_atac/*txt .`

```
setwd("/oak/stanford/groups/khavari/users/mguo123/psych_project/data/GWAS_enrichment")
SNP.all <- read.delim("./resources/LinkedSNP-all-sorted.bed",header=F,stringsAsFactors = F)
DiseaseName <- names(table(SNP.all$V4))

for (tissue in c("Astro" , "H9_D0", "H9_D10", "H9_D28", "SL_D0",  "SLC_D0")){
print(tissue)
file.shuffle <- list.files('./result_diffloop_atac/',pattern=paste0("^",tissue,"(.*?)\\_overlap_all.bed$"),full.names=T)
                                                                                                                                   
                                                                                                                     

tab.shuffle <- matrix(NA,ncol=length(DiseaseName),nrow=length(file.shuffle))

for (i in 1:length(file.shuffle)){
  info <- file.info(file.shuffle[i])
  
  if (info$size != 0){
    
    tmp <- read.delim(file.shuffle[i],header=F,stringsAsFactors = F)
    disease_SNP <- unique(tmp[,4:5])
    tab.tmp <- table(disease_SNP$V4)
    tab.shuffle[i,] <- tab.tmp[match(DiseaseName,names(tab.tmp))]}
}
colnames(tab.shuffle) <- DiseaseName
tab.shuffle <- data.frame(File=file.shuffle,tab.shuffle,check.names = F,stringsAsFactors = F)
write.table(tab.shuffle,file=paste0("./result_diffloop_atac/",tissue,"_shuffle_100Times_actuallastline_diffloop_allATAC_loops.txt"),sep='\t',quote=F,row.names=F)

}

## psycmpra
SNP.psych <- read.delim("./resources/psych_mpra_lib.bed",header=F,stringsAsFactors = F)
SNPs_arr <- names(table(SNP.psych$V4))

for (tissue in c("Astro" , "H9_D0", "H9_D10", "H9_D28", "SL_D0",  "SLC_D0")){
print(tissue)
file.shuffle <- list.files('./result_diffloop_atac/',pattern=paste0("^",tissue,"(.*?)\\_overlap_psychmpra.bed$"),full.names=T)
                                                                                                                                                                                                                                                   
tab.shuffle <- matrix(NA,ncol=length(SNPs_arr),nrow=length(file.shuffle))

for (i in 1:length(file.shuffle)){
  info <- file.info(file.shuffle[i])
  
  if (info$size != 0){
    
    tmp <- read.delim(file.shuffle[i],header=F,stringsAsFactors = F)
    disease_SNP <- unique(tmp[,4])
    tab.tmp <- table(disease_SNP)
    tab.shuffle[i,] <- tab.tmp[match(SNPs_arr,names(tab.tmp))]}
}
colnames(tab.shuffle) <- Disease_SNP
tab.shuffle <- data.frame(File=file.shuffle,tab.shuffle,check.names = F,stringsAsFactors = F)
write.table(tab.shuffle,file=paste0("./result_diffloop_atac/",tissue,"_shuffle_100Times_actuallastline_diffloop_allATAC_loops_psych.txt"),sep='\t',quote=F,row.names=F)

}eend tissue

# ### Need to merge disease names
#   [3] "Alzheimer Dementia"                                                                                                                                                   
#   [4] "Alzheimer Dementia,Cardiovascular Disease,Coronary artery disease,Myocardial Infarction"                                                                              
#   [5] "Alzheimer Dementia,Cardiovascular Disease,Longevity"                                                                                                                  
#   [6] "Alzheimer Dementia,Coronary artery disease,Longevity,Type II diabetes"                                                                                                
#   [7] "Alzheimer Dementia,Crohn’s Disease,Inflammatory Bowel Disease"                                                                                                        
#   [8] "Alzheimer Dementia,Myocardial Infarction"                                                                                                                             
#   [9] "Alzheimer Dementia,Parental Lifespan,Longevity"                                                                                                                       
#  [10] "Alzheimer Dementia,Schizophrenia"                                                                                                                                     
#  [11] "Alzheimer Dementia,Stroke"                                                                                                                                            
#  [12] "Alzheimer Dementia,Systemic Lupus Erythematosus"                                                                                                                      
#  [13] "Alzheimer Dementia,Type II diabetes"                                                                                                                                  
#  [14] "Anxiety"                                                                                                                                                                                                                                                                                                 
#  [25] "Asthma,Longevity"                                                                                                                                                                                                                                                                                                      
#  [31] "Attention Deficit Hyperactivity Disorder"                                                                                                                             
#  [32] "Attention Deficit Hyperactivity Disorder,Bipolar"                                                                                                                     
#  [33] "Attention Deficit Hyperactivity Disorder,Bipolar,Hypertension,Schizophrenia"                                                                                          
#  [34] "Attention Deficit Hyperactivity Disorder,Bipolar,Sarcoidosis,Schizophrenia"                                                                                           
#  [35] "Attention Deficit Hyperactivity Disorder,Bipolar,Schizophrenia"                                                                                                       
#  [36] "Attention Deficit Hyperactivity Disorder,Bipolar,Schizophrenia,Ulcerative colitis"                                                                                    
#  [37] "Attention Deficit Hyperactivity Disorder,Hypertension"                                                                                                                
#  [38] "Attention Deficit Hyperactivity Disorder,Rheumatoid_Arthritis"                                                                                                                                                                                                                                                          
#  [41] "Bipolar"                                                                                                                                                              
#  [42] "Bipolar,Depression"                                                                                                                                                   
#  [43] "Bipolar,Depression,Schizophrenia"                                                                                                                                     
#  [44] "Bipolar,Ischemic Stroke,Stroke"                                                                                                                                       
#  [45] "Bipolar,Schizophrenia"                                                                                                                                                
#  [46] "Bipolar,Schizophrenia,Type I diabetes"                                                                                                                                                                                                                                                                                     
#  [49] "Borderline Personality Disorder"                                                                                                                                                                                                                              
#  [80] "Coronary artery disease,Parental Lifespan,Longevity"                                                                                                                                                                                                                                                 
# [117] "Depression"                                                                                                                                                           
# [118] "Depression,Schizophrenia"                                                                                                                                                                                                                                                                                                   
# [130] "Hypertension,Longevity"                                                                                                                                                                                                                                                                            
# [132] "Hypertension,Schizophrenia"                                                                                                                                                                                                                  
# [138] "Inflammatory Bowel Disease,Schizophrenia"                                                                                                                                                                                                                                                                                   
# [151] "Longevity"                                                                                                                                                                                                                                                                                                
# [160] "Multiple Sclerosis,Schizophrenia"                                                                                                                                                                                                                                                                                   
# [164] "Obsessive Compulsive Disorder"                                                                                                                                        
# [165] "OCD"                                                                                                                                                                  
# [170] "Panic Disorder"                                                                                                                                                       
# [171] "Parental Lifespan,Longevity"                                                                                                                                          
# [172] "Parental Lifespan,Longevity,Myocardial Infarction"                                                                                                                    
# [173] "Post-traumatic Stress Disorder"                                                                                                                                                                                                                                                                  
# [184] "Schizophrenia"                                                                                                                                                        
# [185] "Schizophrenia,Type II diabetes"    
```

# Step 3: make matrix and them calculate empiric pvalues 

In [1]:
import pandas as pd
import os,glob

save_dir = 'gwas_enrichment_shuffle'

## 3A: all diseases 

In [2]:
res_files = glob.glob('gwas_enrichment_shuffle/*loops.txt')
res_files

['gwas_enrichment_shuffle/Astroshuffle_100Times_actuallastline_diffloop_allATAC_loops.txt',
 'gwas_enrichment_shuffle/H9_D0shuffle_100Times_actuallastline_diffloop_allATAC_loops.txt',
 'gwas_enrichment_shuffle/H9_D10shuffle_100Times_actuallastline_diffloop_allATAC_loops.txt',
 'gwas_enrichment_shuffle/SLC_D0shuffle_100Times_actuallastline_diffloop_allATAC_loops.txt',
 'gwas_enrichment_shuffle/H9_D28shuffle_100Times_actuallastline_diffloop_allATAC_loops.txt',
 'gwas_enrichment_shuffle/SL_D0shuffle_100Times_actuallastline_diffloop_allATAC_loops.txt']

In [3]:
pd.read_table(res_files[0])

,File,acne,alopecia areata,Alzheimer Dementia,"Alzheimer Dementia,Cardiovascular Disease,Coronary artery disease,Myocardial Infarction","Alzheimer Dementia,Cardiovascular Disease,Longevity","Alzheimer Dementia,Coronary artery disease,Longevity,Type II diabetes","Alzheimer Dementia,Crohn’s Disease,Inflammatory Bowel Disease","Alzheimer Dementia,Myocardial Infarction","Alzheimer Dementia,Parental Lifespan,Longevity",...,Systemic Lupus Erythematosus,"Systemic Lupus Erythematosus,Type I diabetes",systemic sclerosis,thyroid,Type I diabetes,"Type I diabetes,Type II diabetes","Type I diabetes,Ulcerative colitis",Type II diabetes,Ulcerative colitis,Wegener's Granulomatosis
0,./result_diffloop_atac//Astro_1_overlap_all.bed,970,765,10659,NaN,NaN,NaN,13.0,NaN,NaN,...,7380,54.0,1754,470,4780,62,23.0,13103,2317,265
1,./result_diffloop_atac//Astro_10_overlap_all.bed,995,775,10533,4.0,NaN,4.0,20.0,4.0,5.0,...,7360,53.0,1699,502,4750,61,23.0,13102,2239,265
2,./result_diffloop_atac//Astro_100_overlap_all.bed,980,778,10661,3.0,NaN,2.0,12.0,2.0,3.0,...,7385,54.0,1819,463,4787,58,22.0,13110,2296,269
3,./result_diffloop_atac//Astro_11_overlap_all.bed,962,802,10553,NaN,NaN,NaN,6.0,NaN,NaN,...,7309,54.0,1775,531,4774,58,22.0,13088,2312,264
4,./result_diffloop_atac//Astro_12_overlap_all.bed,987,770,10579,NaN,NaN,NaN,12.0,NaN,NaN,...,7341,50.0,1745,493,4779,59,23.0,12962,2309,267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,./result_diffloop_atac//Astro_96_overlap_all.bed,977,788,10541,3.0,NaN,2.0,10.0,2.0,3.0,...,7505,52.0,1831,506,4776,59,22.0,13020,2303,264
97,./result_diffloop_atac//Astro_97_overlap_all.bed,968,772,10619,4.0,NaN,2.0,9.0,2.0,3.0,...,7374,53.0,1758,468,4753,55,23.0,13160,2288,268
98,./result_diffloop_atac//Astro_98_overlap_all.bed,971,749,10592,NaN,NaN,NaN,NaN,NaN,NaN,...,7327,54.0,1752,517,4766,61,23.0,13087,2289,266
99,./result_diffloop_atac//Astro_99_overlap_all.bed,980,759,10609,NaN,NaN,NaN,20.0,NaN,NaN,...,7376,54.0,1765,502,4805,61,23.0,13061,2294,268


In [25]:
# label = ['control']*100+['actual']
diseases = ['anxiety', 'attent', 'autism',
       'bipolar', 'depress', 'ocd', 'panic', 'personality', 'schizo', 'traum', 'alz','diabetes']
diseases_str = '|'.join(diseases)

tissue_dz_pval_dict = dict()
tissue_dz_or_dict = dict()
for file in res_files:
    tissue = os.path.basename(file).split('shuffle')[0]
    print(tissue)
    df_orig = pd.read_table(file).fillna(0).drop('File',axis=1)
    df_orig.columns = [x.lower() for x in df_orig.columns]
    df = df_orig.div(df_orig.sum(axis=1), axis=0)
    # subselect columns
    diseases_df = pd.DataFrame()
    diseases_df['dz'] = df.columns
    diseases_df['keep'] = diseases_df.dz.str.contains(diseases_str)
    col_to_keep = diseases_df[diseases_df.keep].dz.to_list()
    df = df[col_to_keep]
    
    dz_to_pval = dict()
    dz_to_or = dict()
    for dz in diseases:
        # print(dz)
        dz_cols = [x for x in col_to_keep if dz in x]
        # print(dz_cols)
        dz_series = df[dz_cols].sum(axis=1)
        pval = pd.Series(dz_series[100]<dz_series[:100]).sum()/100
        oddsratio = dz_series[100]/dz_series[:100].mean()
        dz_to_pval[dz] = pval
        dz_to_or[dz] = oddsratio
    
    tissue_dz_pval_dict[tissue] = dz_to_pval
    tissue_dz_or_dict[tissue] = dz_to_or
#     print(tissue)
#     print(dz_to_pval)

Astro
H9_D0
H9_D10


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars


SLC_D0
H9_D28
SL_D0


In [26]:
tissue_dz_pval_df = pd.DataFrame.from_dict(tissue_dz_pval_dict)
display(tissue_dz_pval_df)
tissue_dz_pval_df.to_csv(os.path.join(save_dir, 'tissue_dz_all_df.csv'))

,Astro,H9_D0,H9_D10,SLC_D0,H9_D28,SL_D0
anxiety,1.00,1.0,1.00,1.0,1.00,1.00
attent,1.00,0.0,0.00,0.0,1.00,0.00
autism,0.00,0.0,0.00,0.0,0.00,0.00
bipolar,1.00,1.0,1.00,1.0,0.65,0.00
depress,1.00,1.0,1.00,1.0,1.00,1.00
ocd,1.00,0.0,1.00,1.0,1.00,1.00
panic,0.00,0.0,0.00,0.0,0.00,0.00
personality,0.96,1.0,0.98,1.0,0.00,1.00
schizo,1.00,1.0,1.00,1.0,0.00,0.99
traum,1.00,1.0,1.00,1.0,1.00,1.00


In [28]:
tissue_dz_or_df = pd.DataFrame.from_dict(tissue_dz_or_dict).fillna(1)
display(tissue_dz_or_df)
tissue_dz_or_df.to_csv(os.path.join(save_dir, 'tissue_dz_all_df_or.csv'))

,Astro,H9_D0,H9_D10,SLC_D0,H9_D28,SL_D0
anxiety,0.652862,0.897332,0.599939,0.938890,0.616709,0.799845
attent,0.811267,1.071401,1.094998,1.397831,0.895552,1.129969
autism,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
bipolar,0.894689,0.890273,0.918488,0.986615,0.996284,1.023530
depress,0.543597,0.495244,0.459919,0.598053,0.747482,0.626095
ocd,0.186700,1.285358,0.473552,0.287140,0.140009,0.248928
panic,1.806441,1.778333,1.519907,1.589793,1.534099,1.667131
personality,0.967106,0.810379,0.907928,0.627743,1.736341,0.659070
schizo,0.980903,0.912078,0.980156,0.954094,1.054513,0.988789
traum,0.747771,0.598426,0.280936,0.610429,0.318394,0.448676


In [35]:
tissue_dz_pval_df_long = pd.melt(tissue_dz_pval_df.reset_index(),id_vars='index')
tissue_dz_pval_df_long.columns = ['dz','tissue','pval']
tissue_dz_or_df_long = pd.melt(tissue_dz_or_df.reset_index(),id_vars='index')
tissue_dz_or_df_long.columns = ['dz','tissue','or']
tissue_dz_all_df = tissue_dz_or_df_long.merge(tissue_dz_pval_df_long,on=['dz','tissue'])
display(tissue_dz_all_df)
tissue_dz_all_df.to_csv(os.path.join(save_dir, 'tissue_dz_all_df_long.csv'))

,dz,tissue,or,pval
0,anxiety,Astro,0.652862,1.00
1,attent,Astro,0.811267,1.00
2,autism,Astro,1.000000,0.00
3,bipolar,Astro,0.894689,1.00
4,depress,Astro,0.543597,1.00
...,...,...,...,...
67,personality,SL_D0,0.659070,1.00
68,schizo,SL_D0,0.988789,0.99
69,traum,SL_D0,0.448676,1.00
70,alz,SL_D0,0.765234,1.00


## 3B. psych stuff only


In [64]:
lib_df  = pd.read_csv('~/Google Drive/1_khavari/noncancer_project/psych_lib_info.csv',index_col=0)
lib_df['rowname'] = lib_df.Chr + '_' + lib_df.Position.map(str)
lib_df.columns

Index(['name', 'X', 'Linked_SNP', 'Chr', 'Position', 'Index_SNP', 'Disease',
       'DHS_tissue', 'eGene', 'eQTL_tissue',
       ...
       'dbscSNV.ada_score', 'dbscSNV.rf_score', 'RawScore', 'PHRED',
       'num_disease', 'disease_interest', 'pass_thres_low', 'pass_thres',
       'CADD_score_bool', 'rowname'],
      dtype='object', length=140)

In [65]:
diseases = ['anxiety', 'attent', 'autism',
       'bipolar', 'depress', 'ocd', 'panic', 'personality', 'schizo', 'traum']
dz_to_rsid_dict = {}
for dz in diseases:
    dz_to_rsid_dict[dz] = sorted(set(lib_df[lib_df[dz]==1].Linked_SNP.to_list()))
    print(dz,len(dz_to_rsid_dict[dz]))

anxiety 179
attent 142
autism 54
bipolar 400
depress 424
ocd 116
panic 61
personality 39
schizo 1067
traum 101


In [66]:
res_files = glob.glob('gwas_enrichment_shuffle/*psych.txt')
res_files

['gwas_enrichment_shuffle/SL_D0_shuffle_100Times_actuallastline_diffloop_allATAC_loops_psych.txt',
 'gwas_enrichment_shuffle/H9_D10_shuffle_100Times_actuallastline_diffloop_allATAC_loops_psych.txt',
 'gwas_enrichment_shuffle/H9_D0_shuffle_100Times_actuallastline_diffloop_allATAC_loops_psych.txt',
 'gwas_enrichment_shuffle/Astro_shuffle_100Times_actuallastline_diffloop_allATAC_loops_psych.txt',
 'gwas_enrichment_shuffle/H9_D28_shuffle_100Times_actuallastline_diffloop_allATAC_loops_psych.txt',
 'gwas_enrichment_shuffle/SLC_D0_shuffle_100Times_actuallastline_diffloop_allATAC_loops_psych.txt']

In [67]:

tissue_dz_pval_dict = dict()
tissue_dz_or_dict = dict()
for file in res_files:
    tissue = os.path.basename(file).split('_shuffle')[0]
    print(tissue)
    df_orig = pd.read_table(file).fillna(0).drop('File',axis=1)
    df = df_orig#.div(df_orig.sum(axis=1), axis=0)
    # subselect columns    
    dz_to_pval = dict()
    dz_to_or = dict()
    for dz in diseases:
        # print(dz)
        dz_cols = dz_to_rsid_dict[dz]
        # print(dz_cols)
        dz_series = df[dz_cols].sum(axis=1)
        pval = pd.Series(dz_series[100]<dz_series[:100]).sum()/100
        oddsratio = dz_series[100]/dz_series[:100].mean()
        dz_to_pval[dz] = pval
        dz_to_or[dz] = oddsratio
    
    tissue_dz_pval_dict[tissue] = dz_to_pval
    tissue_dz_or_dict[tissue] = dz_to_or

SL_D0
H9_D10
H9_D0
Astro
H9_D28
SLC_D0


In [68]:
tissue_dz_pval_psych_df = pd.DataFrame.from_dict(tissue_dz_pval_dict)
display(tissue_dz_pval_psych_df)
tissue_dz_pval_psych_df.to_csv(os.path.join(save_dir, 'tissue_dz_pval_psych_df.csv'))

,SL_D0,H9_D10,H9_D0,Astro,H9_D28,SLC_D0
anxiety,1.0,1.00,1.00,1.0,1.0,0.0
attent,0.0,0.98,0.00,1.0,1.0,0.0
autism,1.0,1.00,1.00,1.0,1.0,1.0
bipolar,0.0,1.00,0.00,1.0,1.0,0.0
depress,1.0,1.00,1.00,1.0,1.0,1.0
ocd,0.0,1.00,0.00,1.0,1.0,0.0
panic,0.0,1.00,0.06,1.0,1.0,0.0
personality,1.0,1.00,1.00,1.0,1.0,1.0
schizo,0.0,0.00,0.00,1.0,1.0,0.0
traum,1.0,1.00,1.00,1.0,1.0,1.0


In [69]:
tissue_dz_or_psych_df = pd.DataFrame.from_dict(tissue_dz_or_dict).fillna(1)
display(tissue_dz_or_psych_df)
tissue_dz_or_psych_df.to_csv(os.path.join(save_dir, 'tissue_dz_or_psych_df.csv'))

,SL_D0,H9_D10,H9_D0,Astro,H9_D28,SLC_D0
anxiety,0.762785,0.493131,0.792875,0.399233,0.398865,1.208263
attent,1.978457,0.922070,1.302083,0.414207,0.459982,1.527002
autism,0.798258,0.106045,0.217286,0.018882,0.062999,0.149063
bipolar,2.177569,0.746194,1.604055,0.528642,0.712799,1.824603
depress,0.922971,0.375821,0.547746,0.227542,0.358423,0.645943
ocd,2.495010,0.569106,2.583423,0.390016,0.406872,2.209945
panic,1.268743,0.563456,1.003680,0.610241,0.343014,1.143674
personality,0.640732,0.496483,0.700000,0.408163,0.549451,0.452318
schizo,2.232430,1.182081,1.775361,0.848232,0.824041,2.087499
traum,0.476687,0.169512,0.420839,0.321113,0.033621,0.679579


In [70]:
tissue_dz_pval_psych_df_long = pd.melt(tissue_dz_pval_psych_df.reset_index(),id_vars='index')
tissue_dz_pval_psych_df_long.columns = ['dz','tissue','pval']
tissue_dz_or_psych_df_long = pd.melt(tissue_dz_or_psych_df.reset_index(),id_vars='index')
tissue_dz_or_psych_df_long.columns = ['dz','tissue','or']
tissue_dz_all_psych_df = tissue_dz_or_psych_df_long.merge(tissue_dz_pval_psych_df_long,on=['dz','tissue'])
display(tissue_dz_all_psych_df[tissue_dz_all_psych_df.pval<0.05])
tissue_dz_all_psych_df.to_csv(os.path.join(save_dir, 'tissue_dz_psych_df_long.csv'))

,dz,tissue,or,pval
1,attent,SL_D0,1.978457,0.0
3,bipolar,SL_D0,2.177569,0.0
5,ocd,SL_D0,2.495010,0.0
6,panic,SL_D0,1.268743,0.0
8,schizo,SL_D0,2.232430,0.0
18,schizo,H9_D10,1.182081,0.0
21,attent,H9_D0,1.302083,0.0
23,bipolar,H9_D0,1.604055,0.0
25,ocd,H9_D0,2.583423,0.0
28,schizo,H9_D0,1.775361,0.0


## 3C. psych stuff only mpra sig hits


In [71]:
mpra_res_df = pd.read_csv('D_mpraanalyze_barcode_allelic/final_df_filt.csv',index_col=0)
mpra_res_df = mpra_res_df.merge(lib_df[['Linked_SNP','rowname']],how='left',on='rowname' )
mpra_res_df[:5]

,anxiety,attent,autism,bipolar,depress,ocd,panic,personality,schizo,traum,rowname,GeneName,tissue,pval,fdr,logFC,DHS_tissue,PHRED,ngene,Linked_SNP
0,0,0,0,0,0,0,0,0,1,0,chr4_103750316,UBE2D3,AST1,2.777193e-23,6.129265e-20,4.841936,astrocyte_of_the_hippocampus,5.118,UBE2D3,rs223385
1,0,0,0,0,0,0,0,0,1,0,chr4_103750316,UBE2D3,AST1,2.777193e-23,6.129265e-20,4.841936,astrocyte_of_the_hippocampus,5.118,UBE2D3,rs223385
2,0,0,0,0,0,0,0,0,1,0,chr4_103750316,UBE2D3,AST1,2.777193e-23,6.129265e-20,4.841936,astrocyte_of_the_hippocampus,5.118,UBE2D3,rs223385
3,0,0,0,0,0,0,0,0,1,0,chr4_103750316,UBE2D3,AST1,2.777193e-23,6.129265e-20,4.841936,astrocyte_of_the_hippocampus,5.118,UBE2D3,rs223385
4,0,0,0,0,1,0,0,0,0,0,chr7_12261911,TMEM106B,AST1,4.156023e-22,4.586171e-19,3.258018,"common_myeloid_progenitor,_cd34-positive",2.275,TMEM106B,rs11509880


In [72]:
diseases = ['anxiety', 'attent', 'autism',
       'bipolar', 'depress', 'ocd', 'panic', 'personality', 'schizo', 'traum']
dz_to_rsid_dict = {}
for dz in diseases:
    dz_to_rsid_dict[dz] = sorted(set(mpra_res_df[mpra_res_df[dz]==1].Linked_SNP.to_list()))
    print(dz,len(dz_to_rsid_dict[dz]))

anxiety 56
attent 46
autism 10
bipolar 105
depress 117
ocd 36
panic 15
personality 12
schizo 322
traum 30


In [73]:
res_files = glob.glob('gwas_enrichment_shuffle/*psych.txt')
res_files

['gwas_enrichment_shuffle/SL_D0_shuffle_100Times_actuallastline_diffloop_allATAC_loops_psych.txt',
 'gwas_enrichment_shuffle/H9_D10_shuffle_100Times_actuallastline_diffloop_allATAC_loops_psych.txt',
 'gwas_enrichment_shuffle/H9_D0_shuffle_100Times_actuallastline_diffloop_allATAC_loops_psych.txt',
 'gwas_enrichment_shuffle/Astro_shuffle_100Times_actuallastline_diffloop_allATAC_loops_psych.txt',
 'gwas_enrichment_shuffle/H9_D28_shuffle_100Times_actuallastline_diffloop_allATAC_loops_psych.txt',
 'gwas_enrichment_shuffle/SLC_D0_shuffle_100Times_actuallastline_diffloop_allATAC_loops_psych.txt']

In [74]:
df_orig.sum(axis=1)

0       877.0
1       875.0
2       875.0
3       867.0
4       866.0
        ...  
96      865.0
97      871.0
98      866.0
99      879.0
100    1260.0
Length: 101, dtype: float64

In [75]:

tissue_dz_pval_dict = dict()
tissue_dz_or_dict = dict()
for file in res_files:
    tissue = os.path.basename(file).split('_shuffle')[0]
    print(tissue)
    df_orig = pd.read_table(file).fillna(0).drop('File',axis=1)
    df = df_orig#df_orig.div(df_orig.sum(axis=1), axis=0)
    # subselect columns    
    dz_to_pval = dict()
    dz_to_or = dict()
    for dz in diseases:
        # print(dz)
        dz_cols = dz_to_rsid_dict[dz]
        # print(dz_cols)
        dz_series = df[dz_cols].sum(axis=1)
        pval = pd.Series(dz_series[100]<dz_series[:100]).sum()/100
        oddsratio = dz_series[100]/dz_series[:100].mean()
        dz_to_pval[dz] = pval
        dz_to_or[dz] = oddsratio
    
    tissue_dz_pval_dict[tissue] = dz_to_pval
    tissue_dz_or_dict[tissue] = dz_to_or

SL_D0
H9_D10
H9_D0
Astro
H9_D28
SLC_D0


In [76]:
tissue_dz_pval_psychmpra_df = pd.DataFrame.from_dict(tissue_dz_pval_dict)
display(tissue_dz_pval_psychmpra_df)
tissue_dz_pval_psychmpra_df.to_csv(os.path.join(save_dir, 'tissue_dz_pval_psychmpra_df.csv'))

,SL_D0,H9_D10,H9_D0,Astro,H9_D28,SLC_D0
anxiety,0.97,1.00,1.0,1.00,1.00,0.6
attent,0.00,0.00,0.0,1.00,1.00,0.0
autism,0.00,1.00,1.0,1.00,1.00,1.0
bipolar,0.00,0.81,0.0,1.00,0.59,0.0
depress,0.00,1.00,1.0,1.00,1.00,1.0
ocd,0.00,1.00,0.0,1.00,1.00,0.0
panic,0.00,0.99,0.0,1.00,1.00,0.0
personality,1.00,1.00,1.0,1.00,1.00,1.0
schizo,0.00,0.00,0.0,0.71,1.00,0.0
traum,1.00,1.00,1.0,1.00,1.00,1.0


In [77]:
tissue_dz_or_psychmpra_df = pd.DataFrame.from_dict(tissue_dz_or_dict).fillna(1)
display(tissue_dz_or_psychmpra_df)
tissue_dz_or_psychmpra_df.to_csv(os.path.join(save_dir, 'tissue_dz_or_psychmpra_df.csv'))

,SL_D0,H9_D10,H9_D0,Astro,H9_D28,SLC_D0
anxiety,0.860778,0.406504,0.710419,0.387597,0.216802,0.967622
attent,3.030303,1.101101,1.392758,0.726577,0.631662,1.789137
autism,1.050420,0.123305,0.275482,0.000000,0.000000,0.000000
bipolar,2.805155,0.880065,1.766382,0.763498,0.928232,2.291326
depress,1.231990,0.462107,0.559441,0.245207,0.455451,0.784251
ocd,1.840491,0.395257,2.262931,0.270793,0.270758,1.460920
panic,1.226994,0.735294,1.142857,0.513875,0.271739,1.286174
personality,0.417246,0.112486,0.466563,0.099502,0.128700,0.200200
schizo,2.454765,1.322077,1.829904,0.984804,0.857980,2.312201
traum,0.380590,0.076249,0.274851,0.208420,0.000000,0.459184


In [78]:
tissue_dz_pval_psychmpra_df_long = pd.melt(tissue_dz_pval_psychmpra_df.reset_index(),id_vars='index')
tissue_dz_pval_psychmpra_df_long.columns = ['dz','tissue','pval']
tissue_dz_or_psychmpra_df_long = pd.melt(tissue_dz_or_psychmpra_df.reset_index(),id_vars='index')
tissue_dz_or_psychmpra_df_long.columns = ['dz','tissue','or']
tissue_dz_all_psychmpra_df = tissue_dz_or_psychmpra_df_long.merge(tissue_dz_pval_psychmpra_df_long,on=['dz','tissue'])
display(tissue_dz_all_psychmpra_df)
tissue_dz_all_psychmpra_df.to_csv(os.path.join(save_dir, 'tissue_dz_psychmpra_df_long.csv'))

,dz,tissue,or,pval
0,anxiety,SL_D0,0.860778,0.97
1,attent,SL_D0,3.030303,0.00
2,autism,SL_D0,1.050420,0.00
3,bipolar,SL_D0,2.805155,0.00
4,depress,SL_D0,1.231990,0.00
5,ocd,SL_D0,1.840491,0.00
6,panic,SL_D0,1.226994,0.00
7,personality,SL_D0,0.417246,1.00
8,schizo,SL_D0,2.454765,0.00
9,traum,SL_D0,0.380590,1.00
